In [1]:
# Jupyter Notebook cell: RotatE training and evaluation

import random
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm

c:\Users\jerry\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Load triples
def load_triples(path):
    triples = []
    with open(path, 'r') as f:
        for line in f:
            h, r, t = line.strip().split()[:3]
            triples.append((h, r, t))
    return triples

train_triples = load_triples("../graph_data/train.tsv")
dev_triples =  load_triples("../graph_data/dev.tsv")
test_triples =  load_triples("../graph_data/test.tsv")

# 2. Build vocab
entities = sorted({e for (h, _, t) in train_triples + dev_triples + test_triples for e in (h, t)})
relations = sorted({r for (_, r, _) in train_triples + dev_triples + test_triples})
ent2id = {e: i for i, e in enumerate(entities)}
rel2id = {r: i for i, r in enumerate(relations)}

In [3]:
rel2id

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6}

In [3]:
# 3. Model definition
class RotatEModel(nn.Module):
    def __init__(self, num_ent, num_rel, dim):
        super().__init__()
        self.dim = dim
        # entity embeddings: real and imag parts
        self.ent_re = nn.Embedding(num_ent, dim)
        self.ent_im = nn.Embedding(num_ent, dim)
        # relation embeddings: phase angles
        self.rel_ph = nn.Embedding(num_rel, dim)
        nn.init.xavier_uniform_(self.ent_re.weight)
        nn.init.xavier_uniform_(self.ent_im.weight)
        nn.init.uniform_(self.rel_ph.weight, a=-3.1415, b=3.1415)

    def forward(self, h, r, t):
        re_h = self.ent_re(h)
        im_h = self.ent_im(h)
        re_t = self.ent_re(t)
        im_t = self.ent_im(t)
        ph_r = self.rel_ph(r)
        # compute relation as complex: cos, sin
        re_r = torch.cos(ph_r)
        im_r = torch.sin(ph_r)
        # rotate head
        re_hr = re_h * re_r - im_h * im_r
        im_hr = re_h * im_r + im_h * re_r
        # distance
        diff_re = re_hr - re_t
        diff_im = im_hr - im_t
        # score: negative L1 norm
        score = - (diff_re.abs() + diff_im.abs()).sum(dim=-1)
        return score

# 4. Negative sampling

def negative_sampling(batch, num_entities):
    neg = batch.clone()
    mask = torch.rand(batch.size(0), device=batch.device) < 0.5
    # corrupt head
    num_h = mask.sum().item()
    neg[mask, 0] = torch.randint(0, num_entities, (num_h,), device=batch.device)
    # corrupt tail
    num_t = (~mask).sum().item()
    neg[~mask, 2] = torch.randint(0, num_entities, (num_t,), device=batch.device)
    return neg

# 5. Encoding

def encode(triples):
    return torch.tensor([[ent2id[h], rel2id[r], ent2id[t]] for h, r, t in triples], dtype=torch.long)

train_data = encode(train_triples)
dev_data   = encode(dev_triples)
test_data  = encode(test_triples)

In [4]:
# 8. Training and evaluation functions

def train_epoch(model, loader, optimizer, num_entities):
    model.train()
    total_loss = 0
    total = 0
    for batch in tqdm(loader, desc='Train', leave=False):
        batch = batch[0].to(device)
        pos = batch
        neg = negative_sampling(pos, num_entities)
        pos_score = model(pos[:,0], pos[:,1], pos[:,2])
        neg_score = model(neg[:,0], neg[:,1], neg[:,2])
        loss = - (F.logsigmoid(pos_score).mean() + F.logsigmoid(-neg_score).mean())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * pos.size(0)
        total += pos.size(0)
    return total_loss / total

@torch.no_grad()
def evaluate_ranking(triples_tensor, num_entities, num_neg=100, k_list=(1,3,10)):
    model.eval()
    ranks = []
    for idx in tqdm(range(triples_tensor.size(0)), desc='Eval', leave=False):
        h, r, t = triples_tensor[idx]
        h = h.to(device); r = r.to(device); t = t.to(device)
        pos_score = model(h.unsqueeze(0), r.unsqueeze(0), t.unsqueeze(0))
        scores = [pos_score.item()]
        for _ in range(num_neg):
            if random.random() < 0.5:
                h2 = random.randint(0, num_entities-1); t2 = t.item()
            else:
                h2 = h.item(); t2 = random.randint(0, num_entities-1)
            neg_score = model(
                torch.tensor([h2], device=device),
                r.unsqueeze(0),
                torch.tensor([t2], device=device)
            )
            scores.append(neg_score.item())
        sorted_scores = sorted(scores, reverse=True)
        rank = sorted_scores.index(scores[0]) + 1
        ranks.append(rank)
    ranks = torch.tensor(ranks, dtype=torch.float)
    mrr = (1.0/ranks).mean().item()
    hits = {f"Hits@{k}": (ranks <= k).float().mean().item() for k in k_list}
    return mrr, hits



In [5]:
# 6. Dataloaders
batch_size = 512
train_loader = DataLoader(TensorDataset(train_data), batch_size=batch_size, shuffle=True)
dev_loader   = DataLoader(TensorDataset(dev_data),   batch_size=batch_size)

# 7. Instantiate
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = RotatEModel(len(entities), len(relations), dim=100).to(device)
optimizer = AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)



In [6]:
# 9. Run training

best_dev_loss = float('inf')
for epoch in range(1, 201):
    train_loss = train_epoch(model, train_loader, optimizer, len(entities))
    mrr_dev, hits_dev = evaluate_ranking(dev_data, len(entities))
    print(f"Epoch {epoch:02d} | Train Loss: {train_loss:.4f} | Dev MRR: {mrr_dev:.4f}, Hits@10: {hits_dev['Hits@10']:.4f}")




Epoch 01 | Train Loss: 4.4525 | Dev MRR: 0.0702, Hits@10: 0.1087


Epoch 02 | Train Loss: 3.9407 | Dev MRR: 0.0921, Hits@10: 0.1957


Epoch 03 | Train Loss: 3.5054 | Dev MRR: 0.1399, Hits@10: 0.2609


Epoch 04 | Train Loss: 3.1229 | Dev MRR: 0.1500, Hits@10: 0.2609


Epoch 05 | Train Loss: 2.7869 | Dev MRR: 0.1416, Hits@10: 0.2609


...

Epoch 196 | Train Loss: 0.8130 | Dev MRR: 0.5119, Hits@10: 0.6304


Epoch 197 | Train Loss: 0.8133 | Dev MRR: 0.5330, Hits@10: 0.6304


Epoch 198 | Train Loss: 0.8127 | Dev MRR: 0.5223, Hits@10: 0.6522


Epoch 199 | Train Loss: 0.8132 | Dev MRR: 0.5433, Hits@10: 0.6304


Epoch 200 | Train Loss: 0.8128 | Dev MRR: 0.5468, Hits@10: 0.6304


In [8]:
# 10. Final test evaluation
mrr_test, hits_test = evaluate_ranking(test_data, len(entities))
print(f"Test MRR: {mrr_test:.4f}   " + ", ".join([f"{k}: {v:.4f}" for k,v in hits_test.items()]))


Test MRR: 0.5107   Hits@1: 0.4565, Hits@3: 0.5435, Hits@10: 0.5652
